# Some powerline can be disconnected, and there is ~~nothing~~ something you can do about it
Try me out interactively with: [![Binder](./img/badge_logo.svg)](https://mybinder.org/v2/gh/rte-france/Grid2Op/master)

# test reco maintenance (in jupyter notebook)

In [1]:
import grid2op
import numpy as np
print("grid2op version: {}".format(grid2op.__version__))
env = grid2op.make("l2rpn_wcci_2020", difficulty="0")  # TODO make a test env for that

/home/benjamin/Documents/grid2op_dev/getting_started/grid2op/Backend/PandaPowerBackend.py:38: UserWarning: Numba cannot be loaded. You will gain possibly massive speed if installing it by 
	/home/benjamin/Documents/grid2op_dev/venv/bin/python -m pip install numba

  warnings.warn("Numba cannot be loaded. You will gain possibly massive speed if installing it by "


grid2op version: 0.9.4


In [2]:
env.seed(0)
env.set_id(0)
obs = env.reset()
do_nothing_action = env.action_space()
line_id = 56
start_maintenance = 1259  # first maintenance at 1260 (number of time step) then at 2412
reconnect_action = env.action_space({"set_line_status": [(line_id, +1)]})
# we know (because we made it) that there will be maintenance at ... so we "fast forward until that point"
env.fast_forward_chronics(start_maintenance)
obs = env.get_obs()
print("Can i act on the pwoerline: {}".format(obs.time_before_cooldown_line[line_id] == 0))
print("Is the powerline connected: {}".format(obs.line_status[line_id]))

Can i act on the pwoerline: True
Is the powerline connected: True


In [3]:
# one step pass
obs, reward, done, info = env.step(do_nothing_action)
print("Can i act on the pwoerline: {}".format(obs.time_before_cooldown_line[line_id] == 0))
print("In how many time I will be able to reconnect it: {}".format(obs.time_before_cooldown_line[line_id]))
print("Is the powerline connected: {}".format(obs.line_status[line_id]))

Can i act on the pwoerline: False
In how many time I will be able to reconnect it: 96
Is the powerline connected: False


In [4]:
# let's try to reconnect the powerline
obs, reward, done, info = env.step(reconnect_action)
print("Can i act on the pwoerline: {}".format(obs.time_before_cooldown_line[line_id] == 0))
print("In how many time I will be able to reconnect it: {}".format(obs.time_before_cooldown_line[line_id]))
print("Is the powerline connected: {}".format(obs.line_status[line_id]))
# it has no effect :-(

Can i act on the pwoerline: False
In how many time I will be able to reconnect it: 95
Is the powerline connected: False


In [5]:
# let's then do nothing for 95 remaining time step
# (i can reconnect it only in 95 time steps as said above -> if i try before that it fails...)
for _ in range(95):
    obs, reward, done, info = env.step(do_nothing_action)
print("Can i act on the pwoerline: {}".format(obs.time_before_cooldown_line[line_id] == 0))
print("In how many time I will be able to reconnect it: {}".format(obs.time_before_cooldown_line[line_id]))
print("Is the powerline connected: {}".format(obs.line_status[line_id]))
print("The flow on it is {:.1f}A (of course because it is switched off)".format(obs.a_or[line_id]))

Can i act on the pwoerline: True
In how many time I will be able to reconnect it: 0
Is the powerline connected: False
The flow on it is 0.0A (of course because it is switched off)


In [6]:
# and now reconnect it
obs, reward, done, info = env.step(reconnect_action)
print("Can i act on the powerline: {}".format(obs.time_before_cooldown_line[line_id] == 0))
print("In how many time I will be able to reconnect it: {}".format(obs.time_before_cooldown_line[line_id]))
print("Is the powerline connected: {}".format(obs.line_status[line_id]))
print("The flow on it is {:.1f}A".format(obs.a_or[line_id]))

Can i act on the powerline: True
In how many time I will be able to reconnect it: 0
Is the powerline connected: True
The flow on it is 167.5A
